## Load Checkpoint/the model

In [2]:
import transformers

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [8]:
checkpoint_dir ="/content/drive/MyDrive/Amazon_shoe_review/checkpoint-4221"

In [9]:
#load the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir)

In [10]:
#load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)

## Make Prediction

In [15]:
raw_inputs = [
    "The shoes fell to pieces after a few days, why did I buy them??",
    "This is a very comfortable pair of shoes, I'm super happy with it.",
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")

In [16]:
inputs

{'input_ids': tensor([[ 101, 1996, 6007, 3062, 2000, 4109, 2044, 1037, 2261, 2420, 1010, 2339,
         2106, 1045, 4965, 2068, 1029, 1029,  102],
        [ 101, 2023, 2003, 1037, 2200, 6625, 3940, 1997, 6007, 1010, 1045, 1005,
         1049, 3565, 3407, 2007, 2009, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [24]:
outputs = model(inputs["input_ids"], attention_mask=inputs["attention_mask"])

In [25]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.5638,  1.9136,  0.1254, -2.6943, -3.5762],
        [-2.6418, -2.9041, -1.0317,  2.3485,  3.4736]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

These values are not probabilities. These are actual activations of the five neurons on the last layer of the model. We apply the softmax function to the output logits, and see that we get the same top probability as with the pipeline above.

In [26]:
import torch

In [27]:
probs = torch.softmax(outputs.logits, axis=1)

In [28]:
probs

tensor([[8.1511e-01, 1.5651e-01, 2.6178e-02, 1.5609e-03, 6.4616e-04],
        [1.6486e-03, 1.2683e-03, 8.2492e-03, 2.4233e-01, 7.4650e-01]],
       grad_fn=<SoftmaxBackward0>)

## Push the model to the Hub

In [33]:
from huggingface_hub import notebook_login

In [30]:
notebook_login()

In [31]:
model.push_to_hub("mazed/distilbert-amazon-shoe-review")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mazed/distilbert-amazon-shoe-review/commit/eb21e0e76d2b56f3b5082a59031097bb9462d0d3', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='eb21e0e76d2b56f3b5082a59031097bb9462d0d3', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
tokenizer.push_to_hub("mazed/distilbert-amazon-shoe-review")

CommitInfo(commit_url='https://huggingface.co/mazed/distilbert-amazon-shoe-review/commit/78534ed4d6a77dd20844b4caa126ab04ac972784', commit_message='Upload tokenizer', commit_description='', oid='78534ed4d6a77dd20844b4caa126ab04ac972784', pr_url=None, pr_revision=None, pr_num=None)

## Predict with a Transformer pipeline

In [34]:
from transformers import pipeline

In [36]:
classifier = pipeline("text-classification", model="mazed/distilbert-amazon-shoe-review")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [37]:
classifier("The shoes fell to pieces after a few days, why did I buy them??")

[{'label': 'LABEL_0', 'score': 0.8151060938835144}]

In [39]:
prediction = classifier("This is a very comfortable pair of shoes, I'm super happy with it.")

In [40]:
prediction

[{'label': 'LABEL_4', 'score': 0.7464997172355652}]

In [41]:
prediction[0]

{'label': 'LABEL_4', 'score': 0.7464997172355652}

In [42]:
prediction[0]['label']

'LABEL_4'